In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
from mrmr import mrmr_classif
import catboost

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
selected_features = mrmr_classif(X=X, y=y, K=20)
print("Selected features:", selected_features)

100%|███████████████████████████████████████████████| 20/20 [00:00<00:00, 71.85it/s]

Selected features: ['Follicle_No_R', 'Menstrual_Irregularity', 'Weight_gainY_N', 'Antral_Follicle_Count', 'Testosterone_Levelng_dL', 'Follicle_No_L', 'Skin_darkening_Y_N', 'hair_growthY_N', 'CycleR_I', 'BMI', 'Fast_food_Y_N', 'AMHng_mL', 'Weight_Kg', 'PimplesY_N', 'Hipinch', 'Cycle_lengthdays', 'Waistinch', 'Hair_lossY_N', 'Avg_F_size_L_mm', 'Pulse_ratebpm']


In [8]:
X = X[selected_features]
X

,Follicle_No_R,Menstrual_Irregularity,Weight_gainY_N,Antral_Follicle_Count,Testosterone_Levelng_dL,Follicle_No_L,Skin_darkening_Y_N,hair_growthY_N,CycleR_I,BMI,Fast_food_Y_N,AMHng_mL,Weight_Kg,PimplesY_N,Hipinch,Cycle_lengthdays,Waistinch,Hair_lossY_N,Avg_F_size_L_mm,Pulse_ratebpm
0,-1.123016,0.655651,-1.088774,-0.416878,0.920627,-1.006428,-0.987660,-0.947161,-0.959696,-1.339000,1.306220,-0.909726,-1.585740,-1.404307,-0.860593,0.113918,-1.506265,-1.393480,1.248813,1.599567
1,-0.496611,-0.745031,-1.088774,-0.551288,-0.683707,-1.006428,-0.987660,-0.947161,-0.959696,-0.159445,-1.445879,-1.037723,0.187924,-1.404307,-0.261382,0.113918,-0.850794,-1.393480,0.055162,0.190752
2,2.635411,1.122545,-1.088774,0.154361,0.412233,2.292950,-0.987660,-0.947161,-0.959696,-0.075192,1.306220,0.171135,0.518312,1.414918,0.337829,0.113918,0.460148,1.607292,1.248813,-0.513655
3,-1.436218,0.655651,-1.088774,-0.215264,-0.414796,-1.336366,-0.987660,-0.947161,-0.959696,0.851601,-1.445879,-1.111203,0.187924,-1.404307,0.937040,0.113918,0.460148,-1.393480,0.055162,-0.513655
4,-0.809814,-0.278137,-1.088774,-0.248867,-0.029188,-1.006428,-0.987660,-0.947161,-0.959696,-1.170492,-1.445879,-0.864690,-0.942352,-1.404307,-0.560987,0.113918,-1.506265,1.607292,0.453045,-0.513655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,-0.371331,1.122545,-1.088774,0.927215,1.824777,-0.676490,0.196763,-0.313114,-0.959696,-1.528571,-0.895459,-0.139376,-1.498796,-1.404307,-2.118935,0.113918,-2.358377,-0.793325,0.811141,-0.091011
1537,-0.183409,1.122545,0.451906,-1.760971,-1.584813,0.247336,-0.395448,-0.313114,-0.357909,0.683094,-1.445879,-0.603482,0.196618,-0.276617,0.457671,-0.080918,0.591243,-0.193171,0.214315,-0.020570
1538,-0.371331,-1.211925,-0.061654,1.767273,-1.386935,-0.148590,-0.987660,-0.313114,-0.357909,0.556713,-0.345039,-0.147435,0.568740,-0.840462,0.517592,0.503590,0.460148,-0.193171,0.453045,0.190752
1539,-0.809814,-1.211925,1.479026,-1.424948,1.819703,-0.346552,0.788975,-0.313114,0.845665,0.346078,1.306220,0.040768,0.648729,-0.276617,0.817197,0.113918,1.181167,-0.193171,0.134738,0.190752


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [10]:
from catboost import CatBoostClassifier, Pool

def objective(trial):
    # Define hyperparameters for CatBoost
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 12),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "loss_function": "Logloss",
        "eval_metric": "Accuracy",
        "random_seed": 8,
        "verbose": 0
    }

    # Prepare data
    train_pool = Pool(x_train, y_train)
    valid_pool = Pool(x_test, y_test)

    # Train model
    model = CatBoostClassifier(**param)
    model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=30)

    # Predict
    preds = model.predict(valid_pool)
    acc = accuracy_score(y_test, preds)

    return 1.0 - acc  # Optuna minimizes, so lower is better


In [11]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 21:23:44,505] A new study created in memory with name: no-name-625a2ce7-2cc2-4419-a749-bcf9cc1b1234
[I 2025-05-31 21:23:44,859] Trial 0 finished with value: 0.02155172413793105 and parameters: {'learning_rate': 0.027386070622444497, 'iterations': 780, 'depth': 9, 'subsample': 0.8329986807577601, 'random_strength': 3.844838146303536, 'l2_leaf_reg': 0.7129083165668457}. Best is trial 0 with value: 0.02155172413793105.
[I 2025-05-31 21:23:44,984] Trial 1 finished with value: 0.017241379310344862 and parameters: {'learning_rate': 0.042750400978490706, 'iterations': 439, 'depth': 9, 'subsample': 0.9784414700539205, 'random_strength': 0.37823980347662883, 'l2_leaf_reg': 0.002326354365891704}. Best is trial 1 with value: 0.017241379310344862.
[I 2025-05-31 21:23:45,260] Trial 2 finished with value: 0.025862068965517238 and parameters: {'learning_rate': 0.014662146847892387, 'iterations': 656, 'depth': 10, 'subsample': 0.9164805630796352, 'random_strength': 0.4745506998577688, 'l

Best hyperparameters:
{'learning_rate': 0.027066761755908636, 'iterations': 696, 'depth': 11, 'subsample': 0.8911594301879336, 'random_strength': 1.163342538522754, 'l2_leaf_reg': 0.008427158964939417}
Best accuracy:
0.9956896551724138


In [12]:
# Prepare data
train_pool = Pool(x_train, y_train)
valid_pool = Pool(x_test, y_test)

# Train model
best_model = CatBoostClassifier(**study.best_params)
best_model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=30)

# Predict
preds = best_model.predict(x_test)
preds = (preds > 0.5).astype(int)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")


0:	learn: 0.6145698	test: 0.6261638	best: 0.6261638 (0)	total: 7.58ms	remaining: 5.27s
1:	learn: 0.5541352	test: 0.5725634	best: 0.5725634 (1)	total: 15.1ms	remaining: 5.24s
2:	learn: 0.4944385	test: 0.5149124	best: 0.5149124 (2)	total: 21.6ms	remaining: 4.98s
3:	learn: 0.4477141	test: 0.4814466	best: 0.4814466 (3)	total: 30ms	remaining: 5.18s
4:	learn: 0.4017711	test: 0.4322320	best: 0.4322320 (4)	total: 37.4ms	remaining: 5.17s
5:	learn: 0.3572387	test: 0.3969442	best: 0.3969442 (5)	total: 44.4ms	remaining: 5.1s
6:	learn: 0.3170419	test: 0.3658739	best: 0.3658739 (6)	total: 50.9ms	remaining: 5.01s
7:	learn: 0.2902847	test: 0.3407993	best: 0.3407993 (7)	total: 58.5ms	remaining: 5.03s
8:	learn: 0.2598243	test: 0.3135549	best: 0.3135549 (8)	total: 65.5ms	remaining: 5s
9:	learn: 0.2353875	test: 0.2956378	best: 0.2956378 (9)	total: 72.6ms	remaining: 4.98s
10:	learn: 0.2163044	test: 0.2754676	best: 0.2754676 (10)	total: 79.7ms	remaining: 4.96s
11:	learn: 0.1957362	test: 0.2542504	best: 0.25